# The following code performs the following:
Numerical data: Fills missing values with the median and handles outliers using the IQR method.
Categorical data: Fills missing values with the mode and standardizes text formatting.
Datetime data: Converts date columns and fills missing values with a default fallback date.
Text data: Converts to lowercase and trims whitespace.

In [2]:
import pandas as pd
import numpy as np

def clean_data(df):
    # Drop duplicates
    df = df.drop_duplicates()

    # Identify column types
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
    datetime_cols = [col for col in df.columns if 'date' in col.lower()]

    # Handle missing values
    for col in numerical_cols:
        df[col].fillna(df[col].median(), inplace=True)  # Use median for robustness
    
    for col in categorical_cols:
        df[col].fillna(df[col].mode()[0], inplace=True)
    
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        df[col].fillna(pd.Timestamp('2000-01-01'), inplace=True)  # Default fallback date
    
    # Standardize text data
    for col in categorical_cols:
        df[col] = df[col].astype(str).str.lower().str.strip()
    
    # Handle outliers in numerical columns using IQR method
    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), df[col].median(), df[col])
    
    return df

# Load data from CSV
file_path = "sample_data.csv"  # Update with actual file path
df = pd.read_csv(file_path)

# Clean the data with the updated function
cleaned_df = clean_data(df)

# Save the cleaned dataset
cleaned_file_path = "cleaned_data.csv"  # Update as needed
cleaned_df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned data saved to {cleaned_file_path}")


Cleaned data saved to cleaned_data.csv


# The following code perfroms the data transformation and segmentation, including common operations such as:

Creating new columns
Normalizing numerical values
Encoding categorical variables

Normalizes the "Salary" column.
Encodes the "Department" column.
Categorizes "Age" into groups.

Salary Segmentation: Categorizing salaries into Low, Medium, and High.
Department Segmentation: Assigning the average salary of each department.

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

def transform_data(df):
    # Normalize numerical columns (Scaling Salary)
    scaler = MinMaxScaler()
    df['Salary_Scaled'] = scaler.fit_transform(df[['Salary']])
    
    # Encode categorical columns (Label Encoding Department)
    encoder = LabelEncoder()
    df['Department_Encoded'] = encoder.fit_transform(df['Department'])
    
    # Create a new column (Age Group)
    df['Age_Group'] = pd.cut(df['Age'], bins=[20, 30, 40, 50, 60], labels=['Young', 'Middle-Aged', 'Senior', 'Very Senior'])
    
    return df

def segment_data(df):
    # Segment based on Salary
    df['Salary_Segment'] = pd.cut(df['Salary'], bins=[0, 60000, 80000, 100000], labels=['Low', 'Medium', 'High'])
    
    # Segment based on Department
    department_groups = df.groupby('Department')['Salary'].mean()
    df['Department_Segment'] = df['Department'].map(department_groups)
    
    return df

# Example usage
if __name__ == "__main__":
    # Load sample data
    input_csv = "sample_transformation_data.csv"
    df = pd.read_csv(input_csv)
    
    # Transform and segment the data
    transformed_df = transform_data(df)
    segmented_df = segment_data(transformed_df)
    
    # Save segmented data to a new CSV file
    segmented_csv = "segmented_data.csv"
    segmented_df.to_csv(segmented_csv, index=False)
    
    print(f"Segmented data saved to {segmented_csv}")


Segmented data saved to segmented_data.csv


# Python code example that demonstrates how to fix common data inconsistencies using Pandas. This includes:

Standardizing date formats

Normalizing categorical values (e.g., 'Male', 'male', 'M' → 'Male')

Removing duplicate rows

In [3]:
import pandas as pd

# Sample data with inconsistencies
data = {
    'CustomerID': [101, 102, 103, 104, 105, 105],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Eve'],
    'Gender': ['Male', 'male', 'M', 'Female', 'F', 'FEMALE'],
    'SignupDate': ['01-05-2023', '2023/05/01', 'May 1, 2023', '2023-05-01', '01-May-2023', '2023-05-01']
}

df = pd.DataFrame(data)

print("Original DataFrame:\n", df)

# -------------------------------
# 1. Fixing date format inconsistencies
#df['SignupDate'] = pd.to_datetime(df['SignupDate'], errors='coerce', dayfirst=True)# Converts to datetime, handles mixed formats

# ----------- Fix SignupDate with format parsing ------------
from dateutil import parser

def parse_date(val):
    try:
        return parser.parse(val, dayfirst=True)
    except Exception:
        return pd.NaT

df['SignupDate'] = df['SignupDate'].apply(parse_date)
# -------------------------------
# 2. Standardizing categorical values (Gender column)
def standardize_gender(val):
    val = str(val).strip().lower()
    if val in ['male', 'm']:
        return 'Male'
    elif val in ['female', 'f']:
        return 'Female'
    else:
        return 'Other'

df['Gender'] = df['Gender'].apply(standardize_gender)

# -------------------------------
# 3. Removing duplicate rows (if any)
df = df.drop_duplicates()

print("\nCleaned DataFrame:\n", df)


Original DataFrame:
    CustomerID     Name  Gender   SignupDate
0         101    Alice    Male   01-05-2023
1         102      Bob    male   2023/05/01
2         103  Charlie       M  May 1, 2023
3         104    David  Female   2023-05-01
4         105      Eve       F  01-May-2023
5         105      Eve  FEMALE   2023-05-01

Cleaned DataFrame:
    CustomerID     Name  Gender SignupDate
0         101    Alice    Male 2023-05-01
1         102      Bob    Male 2023-01-05
2         103  Charlie    Male 2023-05-01
3         104    David  Female 2023-01-05
4         105      Eve  Female 2023-05-01
5         105      Eve  Female 2023-01-05
